![Banner](images/banner.png)

# ChinesePod SQLite Database Project
   
This jupyter notebook will read in the data scraped from the ChinesePod website which was obtained as part of my [ChinesePod Project](https://github.com/dlwhittenbury/ChinesePod_Project). The purpose of this project is for me to become more familiar with SQLite databases and their use with python. In this notebook we will 

   - Clean and arrange the data into normalised dataframes/tables using the pandas library
   - Design a relational database to hold the data 
   - Normalise the database
   - Create a SQLite database using python 
   - Load data into the SQLite database using python
   - Query the SQLite database using python

SQLite was chosen over other database management systems, such as MySQL, because the size of the data is relatively small and the database structure is quite simple.


## Imports

In [1]:
import pandas as pd 
import numpy as np
import sqlite3
import os

## Read in data 

Data (INPUT: Path to ChinesePod data)

In [2]:
local_CP_dir = './data'

Load data into dataframe

In [3]:
store = pd.HDFStore(local_CP_dir+'/ChinesePod-Library-Cleaned.h5', 'r')
data = store['data']

## Images folder

In [4]:
# Create directory if doesn't already exist
img_directory = './images'
if not os.path.exists(img_directory):
    os.mkdir(img_directory)
    print("Directory: " , img_directory ,  " created.")

## Look at the data 

Column names

In [5]:
data.columns

Index(['Name', 'Level', 'Series', 'URL', 'Teachers', 'Date', 'Video', 'Tags',
       'Functions', 'ImageURL', 'Description'],
      dtype='object')

The lesson 'Key' was set as the index 

In [6]:
data.index

Index(['QW0465', '4263', '4423', '4272', '4271', '4266', 'JM0001', '4426',
       '4262', '4265',
       ...
       '0033', '0031', '0024', '0001', '0002', '0003', '0004', '0005', '0006',
       'WELCOME'],
      dtype='object', name='Key', length=4305)

In [7]:
data.reset_index(inplace=True)

In [8]:
data.head()

,Key,Name,Level,Series,URL,Teachers,Date,Video,Tags,Functions,ImageURL,Description
0,QW0465,Five ways of seeing: 看、见、看见、看到、看出,Pre Intermediate,Qing Wen,https://chinesepod.com/lesson-redirect?v3id=QW...,"[Joy, Joe]",2020-01-10,True,"[video, synonyms]",[],https://s3contents.chinesepod.com/extra/QW0465...,"11:19 \nIn this Qingwen, we answer a quest..."
1,4263,Animal Proverbs: Become a pig to catch a tiger,Intermediate,Intermediate,https://chinesepod.com/lesson-redirect?v3id=4263,"[Joy, Joshua]",2020-01-08,True,"[video, animal proverbs, animals]",[],https://s3contents.chinesepod.com/4263/400906d...,10:07 \n\nAnimal metaphors are prevalent i...
2,4423,相见恨晚——老电影,Upper Intermediate,Upper Intermediate,https://chinesepod.com/lesson-redirect?v3id=4423,"[Mikkelson, Peony]",2020-01-06,False,"[movies, film]",[],https://s3contents.chinesepod.com/4423/7bbf9e1...,“有心栽花花不开，无心插柳柳成荫。”费尽心思种下的花朵迟迟不开，随手插下的柳枝却长成一片树荫...
3,4272,Character mind map: 客 guest,Elementary,Elementary,https://chinesepod.com/lesson-redirect?v3id=4272,"[Joy, Joe]",2020-01-03,True,"[video, Character Mind Map, character]",[],https://s3contents.chinesepod.com/4272/733551a...,"9:15 \n\n\nIn Chinese, you often encounter..."
4,4271,How To Use Action Complete “le” 了,Pre Intermediate,Pre Intermediate,https://chinesepod.com/lesson-redirect?v3id=4271,"[Joshua, Jessica]",2019-12-31,True,"[video, grammar, le, actions]",[],https://s3contents.chinesepod.com/4271/7665f59...,"10:28 \nIn this lesson, we're learning the..."


In [9]:
data['Level'].value_counts()

Intermediate          1031
Elementary             994
Upper Intermediate     639
Newbie                 498
Advanced               463
Other                  461
Media                  182
Pre Intermediate        37
Name: Level, dtype: int64

## Normalise the data 

### Teachers dataframe/table 

In [10]:
teachers = data[['Key','Teachers']]
teachers.head()

,Key,Teachers
0,QW0465,"[Joy, Joe]"
1,4263,"[Joy, Joshua]"
2,4423,"[Mikkelson, Peony]"
3,4272,"[Joy, Joe]"
4,4271,"[Joshua, Jessica]"


In [11]:
teachers = teachers.assign(Teachers=teachers.Teachers).explode('Teachers').reset_index(drop=True)

In [12]:
teachers = teachers.rename(columns={'Teachers':'Teacher'})

In [13]:
teachers.head()

,Key,Teacher
0,QW0465,Joy
1,QW0465,Joe
2,4263,Joy
3,4263,Joshua
4,4423,Mikkelson


Strip whitespace

In [14]:
teachers['Teacher'] = teachers['Teacher'].str.strip()

We will drop null values 

In [15]:
teachers.dropna(inplace=True)

Check for duplicates 

In [16]:
teachers[teachers.duplicated()]

,Key,Teacher
1235,2815,Gwilym
1236,2815,Fiona Tian
1429,2733,Gwilym
1430,2733,Fiona Tian
1452,2724,Gwilym
1453,2724,Fiona Tian
1778,2583,Ryan
1779,2583,Jenny Zhu
1831,2560,Jenny Zhu
1832,2560,Ryan


In [17]:
teachers.drop_duplicates(keep='first',inplace=True)

In [18]:
teachers[teachers.duplicated()]

,Key,Teacher


In [19]:
teachers.head()

,Key,Teacher
0,QW0465,Joy
1,QW0465,Joe
2,4263,Joy
3,4263,Joshua
4,4423,Mikkelson


In [20]:
teachers.to_csv(os.path.join(local_CP_dir,'Teachers.csv'),index=False,sep=',',header=True)

In [21]:
teachers[teachers['Teacher'].str.contains('Jenny Zhu')]

,Key,Teacher
245,BOG0003,Jenny Zhu
297,4122,Jenny Zhu
305,4113,Jenny Zhu
307,QW0460,Jenny Zhu
309,4114,Jenny Zhu
...,...,...
8907,0003,Jenny Zhu
8909,0004,Jenny Zhu
8911,0005,Jenny Zhu
8913,0006,Jenny Zhu


### Tags dataframe/table

In [22]:
tags = data[['Key','Tags']]
tags.head()

,Key,Tags
0,QW0465,"[video, synonyms]"
1,4263,"[video, animal proverbs, animals]"
2,4423,"[movies, film]"
3,4272,"[video, Character Mind Map, character]"
4,4271,"[video, grammar, le, actions]"


In [23]:
tags = tags.assign(Tags=tags.Tags).explode('Tags').reset_index(drop=True)

In [24]:
tags = tags.rename(columns={'Tags':'Tag'})

Strip whitespace 

In [25]:
tags['Tag'] = tags['Tag'].str.strip()

In [26]:
tags.head() 

,Key,Tag
0,QW0465,video
1,QW0465,synonyms
2,4263,video
3,4263,animal proverbs
4,4263,animals


We will drop null values 

In [27]:
tags.dropna(inplace=True)

Check for duplicates

In [28]:
tags[tags.duplicated()]

,Key,Tag


In [29]:
tags.to_csv(os.path.join(local_CP_dir,'Tags.csv'),index=False,sep=',',header=True)

### Functions dataframe/table 

In [30]:
functions = data[['Key','Functions']]

In [31]:
functions = functions.assign(Functions=functions.Functions).explode('Functions').reset_index(drop=True)

In [32]:
functions = functions.rename(columns={'Functions':'Function'})

Strip whitespace 

In [33]:
functions['Function'] = functions['Function'].str.strip()

We will drop null values 

In [34]:
functions.dropna(inplace=True)

In [35]:
functions.head()

,Key,Function
0,QW0465,
1,4263,
2,4423,
3,4272,
4,4271,


Check for duplicates

In [36]:
functions[functions.duplicated()]

,Key,Function


In [37]:
functions.replace('', np.nan, inplace=True)
functions.dropna(inplace=True)

In [38]:
functions.head()

,Key,Function
157,4123,character
158,4123,reading
159,4100,dealing with praise
160,4100,appreciating
163,4121,finding


In [39]:
functions.to_csv(os.path.join(local_CP_dir,'Functions.csv'),index=False,sep=',',header=True)

### Main part of the data will be stored in lessons dataframe/table 

In [40]:
lessons = data.drop(columns=['Teachers','Tags','Functions'])
lessons.columns.values

array(['Key', 'Name', 'Level', 'Series', 'URL', 'Date', 'Video',
       'ImageURL', 'Description'], dtype=object)

In [41]:
lessons.to_csv(os.path.join(local_CP_dir,'Lessons.csv'),index=False,sep=',',header=True)

### Excel file 

In [42]:
with pd.ExcelWriter(os.path.join(local_CP_dir,'ChinesePod.xlsx')) as writer:  
    lessons.to_excel(writer, sheet_name='Lessons',index=False,header=True)
    teachers.to_excel(writer, sheet_name='Teachers',index=False,header=True)
    tags.to_excel(writer,sheet_name='Tags',index=False,header=True)
    functions.to_excel(writer,sheet_name='Functions',index=False,header=True)

## Logical schema 

The logical schema (created using yEd) for the normalised SQLite database is shown below 

![ChinesePod SQLite database logical schema.](images/logical_schema.png)

## SQLite database 

### Change dates to strings 

SQLite cannot handle dates, so we change it to a formatted string here. 

In [43]:
lessons['Date'] = lessons['Date'].apply(lambda x: x.strftime('%Y-%m-%d'))

### Change the column Video from Boolean to string

In [44]:
lessons.Video = lessons.Video.astype(str)

In [45]:
lessons.Video.head()

0     True
1     True
2    False
3     True
4     True
Name: Video, dtype: object

### Create the database 

Create the database to by making a connection to a non-existent database called 'ChinesePod.db'. If it already exists it won't create it, it will just connect to it.

In [46]:
ChinesePod_db = 'ChinesePod.db'
conn = sqlite3.connect(os.path.join(local_CP_dir,ChinesePod_db))

### Create the tables 

Please note that SQLite has a limited set of data types, see the [documentation](https://www.sqlite.org/datatype3.html).

In [47]:
myCursor = conn.cursor()

In [48]:
create_Lessons_table = """CREATE TABLE IF NOT EXISTS Lessons (
Key TEXT NOT NULL,
Name TEXT,
Level TEXT,
Series TEXT,
URL TEXT,
Date TEXT, 
Video TEXT, 
ImageURL TEXT,
Description TEXT,
PRIMARY KEY (Key)
);"""

In [49]:
myCursor.execute(create_Lessons_table)

In [50]:
create_Teachers_table = """CREATE TABLE IF NOT EXISTS Teachers (
Key TEXT NOT NULL, 
Teacher TEXT NOT NULL,
PRIMARY KEY (Key,Teacher),
FOREIGN KEY (Key) REFERENCES Lessons(Key)
);"""

In [51]:
myCursor.execute(create_Teachers_table)

In [52]:
create_Tags_table = """CREATE TABLE IF NOT EXISTS Tags (
Key TEXT NOT NULL,
Tag TEXT NOT NULL,
PRIMARY KEY (Key,Tag),
FOREIGN KEY (Key) REFERENCES Lessons(Key)
);"""

In [53]:
myCursor.execute(create_Tags_table)

In [54]:
create_Functions_table = """CREATE TABLE IF NOT EXISTS Functions (
Key TEXT NOT NULL, 
Function TEXT NOT NULL, 
PRIMARY KEY (Key,Function),
FOREIGN KEY (Key) REFERENCES Lessons(Key)
);"""

In [55]:
myCursor.execute(create_Functions_table)

## Insert data into tables 

Insert a lesson data 

In [56]:
def insert_lessons(cursor,conn,lessons):
      
    with conn:
    
        for i in range(len(lessons)):

            lesson_dic = lessons.iloc[i].to_dict()
        
            query = """INSERT INTO Lessons(Key,Name,Level,Series,URL,Date,Video,ImageURL,Description) 
            VALUES (:Key,:Name,:Level,:Series,:URL,:Date,:Video,:ImageURL,:Description)"""

            cursor.execute(query,lesson_dic)
    


In [57]:
insert_lessons(myCursor,conn,lessons)

Insert teachers data 

In [58]:
def insert_teachers(cursor,conn,teachers):
    
    with conn: 
        
        for i in range(len(teachers)):
            
            teachers_dic = teachers.iloc[i].to_dict() 
            
            query = """INSERT INTO Teachers(Key,Teacher) VALUES(:Key,:Teacher)"""
            
            cursor.execute(query,teachers_dic)

In [59]:
insert_teachers(myCursor,conn,teachers)

Insert tags data 

In [60]:
def insert_tags(cursor,conn,tags):
    
    with conn: 
        
        for i in range(len(tags)):
            
            tags_dic = tags.iloc[i].to_dict() 
            
            query = """INSERT INTO Tags(Key,Tag) VALUES(:Key,:Tag)"""
            
            cursor.execute(query,tags_dic)

In [61]:
insert_tags(myCursor,conn,tags)

Insert functions data 

In [62]:
def insert_functions(cursor,conn,functions):
    
    with conn: 
        
        for i in range(len(functions)):
            
            functions_dic = functions.iloc[i].to_dict() 
            
            query = """INSERT INTO Functions(Key,Function) VALUES(:Key,:Function)"""
            
            cursor.execute(query,functions_dic)

In [63]:
insert_functions(myCursor,conn,functions)

## Practice querying the new SQLite database ChinesePod.db

### Q1: What are the top 10 most common lesson tags?

In [64]:
Q1 = """SELECT Tag, COUNT(*) AS Count FROM Tags GROUP BY Tag ORDER BY Count DESC LIMIT 10"""

In [65]:
myCursor.execute(Q1)

In [66]:
Q1_data = myCursor.fetchall()

In [67]:
Q1_df = pd.DataFrame(Q1_data,columns=['Tag','Count'])
Q1_df

,Tag,Count
0,video,349
1,culture,291
2,food,271
3,business,193
4,family,172
5,travel,157
6,shopping,152
7,relationships,132
8,people,130
9,time,124


### Q2: Who are the 10 teachers that appeared in the most lessons? 

In [68]:
Q2 = """SELECT Teacher, COUNT(*) AS Count FROM Teachers GROUP BY Teacher ORDER BY Count DESC LIMIT 10"""

In [69]:
myCursor.execute(Q2)

In [70]:
Q2_data = myCursor.fetchall()

In [71]:
Q2_df = pd.DataFrame(Q2_data,columns=['Teacher','Count'])

In [72]:
Q2_df

,Teacher,Count
0,Jenny Zhu,1802
1,John,1484
2,Fiona Tian,654
3,Ken,622
4,Connie,540
5,Dilu,419
6,Jenny,402
7,David,394
8,Constance Fang,356
9,Gwilym,279


### Q3: Which teachers starred in video lessons?

In [73]:
Q3 = """SELECT DISTINCT T.Teacher, COUNT(*) AS Count 
FROM Teachers AS T, Lessons AS L
WHERE T.Key = L.Key 
AND L.Video = 'True'
GROUP BY T.Teacher 
ORDER BY Count DESC 
"""

In [74]:
myCursor.execute(Q3)

In [75]:
Q3_data = myCursor.fetchall() 

In [76]:
Q3_df = pd.DataFrame(Q3_data,columns=['Teacher','Count'])

In [77]:
Q3_df

,Teacher,Count
0,Fiona Tian,205
1,Constance Fang,124
2,Gwilym,71
3,Joy,45
4,Grace,33
5,Joe,29
6,Joshua,23
7,Jenny Zhu,23
8,Sophie,13
9,Jessica,10


## Close database connection 

In [78]:
myCursor.close()
conn.close()